In [5]:
from stock_prediction.data_prep import Data_Prep
from stock_prediction.features_exo import exo_selection

In [33]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import os
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error as MAPE
import itertools
import pdb

<ipython-input-33-870a85ae3975>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [13]:
# instantiate the Class 
vinci = Data_Prep('vinci', 10)

In [14]:
# load data 
data_global = vinci.data_prep()

In [15]:
# select fetarures 
data_exo = vinci.select_features(data_global, Return = True, Log_Return=False, High_Low=True, High_Close=True, Low_Close=True,
                        Volume_Change=False, Period_Volum=True, Annual_Vol=False,
                        Period_Vol=False, Return_Index=True, Volum_Index=True, Relative_Return=False)


In [ ]:
# add exogenous features 
data_exo = exo_selection(data_return, sp500=True, gold=True, eurusd=True, nasdaq=True, crude=True, vix=True)

In [16]:
# select first year 
data_exo = data_exo[-258:]

In [17]:
# rebase the indices 
data_exo = vinci.Price_Rebase(data_exo)

In [18]:
data_exo.columns

Index(['Date', 'High-Low_DG.PA', 'High-Close_DG.PA', 'Low-Close_DG.PA',
       'Period_Volum_DG.PA', 'Period_Volum_stoxx_50', 'Return_DG.PA_R',
       'Return_stoxx_50_R'],
      dtype='object')

In [ ]:
# create the y
y_vinci = data_exo['Return_DG.PA_R']
y_sp = data_exo['Return_S&P500_R']
y_usd = data_exo['Return_Usd_R']
y_nasdaq = data_exo['Return_Nasdaq_R']
y_crude = data_exo['Return_Crude_R']
y_vix = data_exo['Vix_No_Rebase']

In [29]:
# assign the exogenous features 
y_endo = data_exo['Return_DG.PA_R']
y_exo = np.array(data_exo[['High-Low_DG.PA', 'High-Close_DG.PA', 'Low-Close_DG.PA', 
                  'Period_Volum_DG.PA', 'Period_Volum_stoxx_50', 'Return_stoxx_50_R']])

In [32]:
y_exo

array([[ 2.58404073e-02,  1.44730775e-02,  1.12051568e-02,
         1.12379056e-01,  2.10500239e-01,  1.00000000e+02],
       [ 2.70996407e-02,  1.55004539e-02,  1.14221385e-02,
         4.37202976e-01,  4.44184569e-01,  1.03759428e+02],
       [ 3.16957565e-02,  1.71875115e-02,  1.42630978e-02,
        -1.92506074e-02,  9.35392610e-02,  1.03207563e+02],
       ...,
       [ 1.15792970e-02,  5.11340135e-03,  6.43300117e-03,
        -1.78402621e-01, -1.34404909e-01,  1.25349900e+02],
       [ 1.65049196e-02,  8.34677394e-03,  8.09061515e-03,
        -1.63415567e-01, -1.29851946e-01,  1.25065996e+02],
       [ 7.63283190e-03,  2.99631898e-03,  4.62266194e-03,
        -3.55489341e-01, -1.10168466e-01,  1.25376877e+02]])

In [ ]:
model_results_max = []


# we search all the possibilities for max_train 
    
for max_train in range(10,35,1) :

    splits = len(y_endo) - max_train

    # reset the lists for next for max_train
    mapeS = []
    base_score = []
    order = (2,1,0)
    # make the split of the indexes of our TS
    folds = TimeSeriesSplit(n_splits=splits, max_train_size=max_train, test_size=1)
    # fold_idx is the number of the split -- n_splits ?  
    # train_idx are the indexes of the split n°X
    # same for test_idx

    for train_idx, test_idx in folds.split(y_endo):

        # we retrieve the real data in y 
        #corresponding of the indexes splited 
        y_train = y_endo[train_idx]
        y_exo_train = y_exo[train_idx]
        y_test = y_endo[test_idx]
        y_exo_test = y_exo[test_idx]
        y_true = y_endo[test_idx]
        base = y_train.iloc[-1]
        #pdb.set_trace()
        # fit our model on y_train of the split n°X
        model = ARIMA(endog=y_train, exog=y_exo_train, order=order).fit()
        # we find our y_pred 
        y_pred = model.forecast(steps=len(y_test), exog=y_exo_test)[0]
        mapeS.append(MAPE(y_pred, y_true))
        # need np.array to compute mean
        base_score.append(MAPE(np.array([base]), y_true))
        mapeS_m = np.array(mapeS)
        base_score_m = np.array(base_score)
        # we store all the results 

        model_results_max.append([max_train, mapeS_m.mean(), base_score_m.mean()])

In [ ]:
model_results_max = pd.DataFrame(model_results_max, columns = ['max_train', 'MAPE', 'MAPE_base'])

In [ ]:
# we show the best max_train regarding our MAPE
model_results_max.head().sort_values(by='MAPE')